In [1]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON
import rdflib
from operator import itemgetter
%load_ext ipython_sparql_pandas

def load_graph(graph, filename):
    with open(filename, 'r') as f:
        graph.parse(f, format='turtle')
        
g = Graph()
load_graph(g, 'skincare_ontology_final.ttl')

def strip(e):
    if type(e) is Literal:
        return e
    if "#" in e:
        wrap = e.split("#")
        n=len(wrap)
        p=wrap[n-1]
        return p
    wrap = e.split("/")
    n=len(wrap)
    p=wrap[n-1]
    return p

In [15]:
#INPUT INTERFACE

#Pick your skintype from [Dry, Oily, Combination, Normal]
a="Oily"
#Pick your skintone from [Light, Fair, Medium, Olive, Tan, Porcelain, Deep, Dark, Ebony]
b="Ebony"
#Pick from [1,2,3] depending on how many steps do you want your recommended routine to be
step=3
#Pick from [Australia, Bahrain, Belgium, Brazil, Bulgaria, Canada, China, Czech_Republic,
# Denmark, Estonia, France, Germany, Greece, Hong_Kong, Hungary, Indonesia, Ireland, Italy, 
# Kuwait, Latvia, Lithuania, Luxembourg, Malaysia, Netherlands, New_Zealand, Oman, Philippines,
#Poland, Portugal, Qatar, Russia, Saudi_Arabia, Singapore, Slovakia, Slovenia, Spain, Sweden, 
#Switzerland, Thailand, Turkey, United_Arab_Emirates, United_Kingdom, United_States]
sephora_variable="Australia"

In [16]:
#input restructuring
itype="http://www.semanticweb.org/saraabesova/ontologies/skincare#"+a 
itone="http://www.semanticweb.org/saraabesova/ontologies/skincare#"+b

steps_list=['Cleanser','Moisturizer','Treatment']
i=0
while i < step:
    print("Top 3 "+steps_list[i]+"s we found for you!")
    products=[]
    result_list=[]

    product_results =g.query("""
        PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

        SELECT DISTINCT ?product
        WHERE { 
            ?reviewID skc:aboutProduct ?product.
            ?product rdf:type rdf:"""+steps_list[i]+""".
        }
        """)
    for result in product_results:
        products.append(result[0])

    for product in products:
        chars = ["®", "/", "’", "–", "°", '„', '¨', '“', '©'] 
        if any([char in strip(product) for char in chars]):
            continue

        ratings=[]
        rating_results =g.query("""
            PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

            SELECT ?rating
            WHERE { 
                ?reviewID skc:hasSkinType <"""+itype+""">. 
                ?reviewID skc:hasSkinTone <"""+itone+""">.
                ?reviewID skc:hasRating ?rating.
                ?reviewID skc:aboutProduct skc:"""+strip(product)+""".
            }
            """)
        for result in rating_results:
            ratings.append(int(result[0])) ###change to int
        if len(rating_results) !=0:
            average= sum(ratings)/len(ratings)
        else:
            continue

        result_list.append([strip(product),average])

    sorted_list=sorted(result_list, key=itemgetter(1), reverse=True)
    
    if len(sorted_list) < 3:
        t =g.query("""
            PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

            SELECT ?product
            WHERE { 
                ?product rdf:type skc:"""+a+"""_Skin_Products.
                ?product rdf:type rdf:"""+steps_list[i]+""".
            }
            """)
        for j in t:
            sorted_list.append(j)

    recommended_list=[]
    recommended_list.append(sorted_list[0])
    recommended_list.append(sorted_list[1])
    recommended_list.append(sorted_list[2])
    
    print(recommended_list[0][0])
    print(recommended_list[1][0])
    print(recommended_list[2][0])
    print("")
    i=i+1

    
#Sephora Online and Physical Availability
isephora = "http://dbpedia.org/resource/"+sephora_variable

webpage_results =g.query("""
    PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
    PREFIX dbr: <http://dbpedia.org/resource/>

    SELECT ?webpage
    WHERE{
        <"""+isephora+"""> skc:hasSephoraWebPage ?webpage.
    } """)
# Sephora is available online
for result in webpage_results:
    webpage_link = str(result[0])
    if webpage_link:
        print(f"The link to Sephora in your country is here: {webpage_link}")
    
physical_stores =g.query("""
    PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
    PREFIX dbr: <http://dbpedia.org/resource/>

    SELECT ?country ?webpage
    WHERE {
        {?country skc:hasSephoraWebPage ?webpage.
            FILTER CONTAINS (str(?country), "France")}
        UNION {?country skc:hasSephoraWebPage ?webpage.
            MINUS {?country skc:hasSephoraWebPage "https://www.sephora.fr/".}
            MINUS {?country skc:hasSephoraWebPage "https://www.sephora.fr/?country_switch=ca&lang=en".}}
}  """)

# Country link DBpedia
physical_stores_list = []
country =  isephora 


# The physical Sephora store is in the country
for result in physical_stores:
    physical_store = str(result[0])
    physical_stores_list.append(physical_store)

    if country == physical_store:
        print("You can also visit the physical Sephora store in your country")

# If the physical Sephora store is not available in the country
if isephora not in physical_stores_list: 
    # and when there is no webpage results
    if len(webpage_results) == 0:
        print("Unfortunately, Sephora is not available in your country.\n" 
              "Try other shops or websites with international shipping:\n"
              "http://www.amazon.com, http://skinstore.com, https://www.cultbeauty.com/ or http://ebay.com")
    else:
        print("Unfortunately, there is no physical Sephora store in your country")

Top 3 Cleansers we found for you!
Clear%20Complexion%20Cleanser
Essential-C%20Toner
Rinse-Off%20Foaming%20Cleanser

Top 3 Moisturizers we found for you!
Water%20Drench%20Hyaluronic%20Cloud%20Cream
http://www.semanticweb.org/saraabesova/ontologies/skincare#CeraVeMoisturizingCream
http://www.semanticweb.org/saraabesova/ontologies/skincare#PaulasChoiceSunscreenSPF50

Top 3 Treatments we found for you!
Acne%20Body%20Wash
Breakout%20Fighters
Protein%20Booster%20Skin%20Serum

The link to Sephora in your country is here: https://www.sephora.com.au/
You can also visit the physical Sephora store in your country


In [4]:
skin = Graph()
SKC = Namespace("http://www.semanticweb.org/saraabesova/ontologies/skincare#")
skin.bind("skc", SKC)

#hasOilyScore
results =g.query("""
    PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?product (AVG(xsd:integer(?rating)) AS ?average)
    WHERE { 
        ?reviewID skc:aboutProduct ?product.
        ?reviewID skc:hasSkinType skc:Oily.
        ?reviewID skc:hasRating ?rating.
    } GROUP BY ?product
    """)

for i in results:
    skin.add((i[0], skc.hasOilyScore, i[1]))

#hasDryScore
results =g.query("""
    PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?product (AVG(xsd:integer(?rating)) AS ?average)
    WHERE { 
        ?reviewID skc:aboutProduct ?product.
        ?reviewID skc:hasSkinType skc:Dry.
        ?reviewID skc:hasRating ?rating.
    } GROUP BY ?product
    """)

for i in results:
    skin.add((i[0], skc.hasDryScore, i[1]))

#hasCombinationScore
results =g.query("""
    PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?product (AVG(xsd:integer(?rating)) AS ?average)
    WHERE { 
        ?reviewID skc:aboutProduct ?product.
        ?reviewID skc:hasSkinType skc:Combination.
        ?reviewID skc:hasRating ?rating.
    } GROUP BY ?product
    """)

for i in results:
    skin.add((i[0], skc.hasCombinationScore, i[1]))
#hasNormalScore
results =g.query("""
    PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?product (AVG(xsd:integer(?rating)) AS ?average)
    WHERE { 
        ?reviewID skc:aboutProduct ?product.
        ?reviewID skc:hasSkinType skc:Normal.
        ?reviewID skc:hasRating ?rating.
    } GROUP BY ?product
    """)

for i in results:
    skin.add((i[0], SKC.hasNormalScore, i[1]))

# serialize_graph(skin)
save_graph(skin, "extra_score_triples.ttl")

NameError: name 'skc' is not defined

In [ ]:
%pip install ipython-sparql-pandas
# run `sparql?` in a cell for documentation
%load_ext ipython_sparql_pandas

In [ ]:
%%sparql http://localhost:7200/repositories/Skincare_Project -s skincare
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>

SELECT ?skin_type (COUNT(*) as ?skin_type_count){ 
    ?reviewID skc:hasSkinType ?skin_type.
} GROUP BY ?skin_type ORDER BY DESC(?skin_type_count)




In [ ]:
skincare["skin_type_count"].plot.pie(
    labels=["Combination", "No%20data", "Dry", "Oily", "Normal"],
    colors=["lightcoral", "darkgrey", "lightsteelblue", "lemonchiffon", "rosybrown"],
    subplots=True,
    fontsize=10,
    figsize=(10, 10),
);

In [ ]:
%%sparql http://localhost:7200/repositories/Skincare_Project -s skincare
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
        
SELECT ?skin_tone (COUNT(*) as ?skin_tone_count){ 
    ?reviewID skc:hasSkinTone ?skin_tone.
} GROUP BY ?skin_tone ORDER BY DESC(?skin_tone_count)


In [ ]:
skincare["skin_tone_count"].plot.pie(
    labels=["No%20data", "Light", "Fair", "Medium", "Olive", "Tan", "Porcelain", "Deep", "Dark", "Ebony"],
    colors=["darkgrey", "seashell", "mistyrose", "peachpuff", "burlywood", "peru", "snow", "sienna", "saddlebrown", "black"],
    subplots=True,
    fontsize=10,
    figsize=(10, 10),
);

In [ ]:
%%sparql http://localhost:7200/repositories/Skincare_Project -q -s sephoraDataFrame
PREFIX skc: <http://www.semanticweb.org/saraabesova/ontologies/skincare#>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?country ?capital ?lat ?long 
WHERE {   
    ?country skc:hasSephoraWebPage ?webpage.
    SERVICE <https://dbpedia.org/sparql> {
        
        ?country dbo:capital ?capital.
        ?capital geo:lat ?lat;
                 geo:long ?long.
        OPTIONAL{?country geo:lat ?lat;
                          geo:long ?long.}
        
    }
}

In [ ]:
%pip install folium

In [ ]:
import folium, math

middle = [sephoraDataFrame['lat'].mean(), sephoraDataFrame['long'].mean()]
m = folium.Map(location=middle, zoom_start=4)
for _, row in sephoraDataFrame.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        color="#000000",
        fill=True,
        fill_color="#F8F8FF",
    ).add_to(m)

m